# Metrics

## Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import yaml

## Load data

In [2]:
true_df = pd.read_csv('../data/videos.csv')
true_df

,filename,classname
0,I_CIDIMEC_Background_1.mp4,Background
1,I_CIDIMEC_Background_2.mp4,Background
2,I_CIDIMEC_Background_3.mp4,Background
3,I_CIDIMEC_Background_4.mp4,Background
4,I_CIDIMEC_Gun_1.mp4,Gun
5,I_CIDIMEC_Gun_2.mp4,Gun
6,I_CIDIMEC_Gun_3.mp4,Gun
7,I_CIDIMEC_Knife_1.mp4,Knife
8,I_CIDIMEC_Knife_2.mp4,Knife
9,I_CIDIMEC_Knife_3.mp4,Knife


## Version 1

### Modules

In [3]:
from utils.image_preprocessing import ImagePreprocessor
from utils.detect import Detect

In [10]:
PATH = "/media/rodri/Files/Datasets/Evaluation videos/Evaluation_V1"
detect = Detect()

### Generate csv

In [12]:
# Show videos
import cv2
import os
import time
from tqdm import tqdm
import yaml

fps_sum = 0
count = 0

def get_params(video: str) -> dict:
    """ This function returns the parameters of the video.

    Args:
        video (str): The name of the video which will be converted on the config file path.
            Example: "I_CIDIMEC_Background_1.mp4" -> "config/inside cidimec.yml"

    Returns:
        dict: The parameters of the video.
    """
    place, background, _ = video.split("_")
    
    place = "inside" if place == "I" else "outside"
    
    scenario = f"{place} {background}"
    
    with open(f"config/{scenario}.yml", 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
        
    return cfg

for video in tqdm(true_df['filename']):
    cap = cv2.VideoCapture(os.path.join(PATH, video))
    image_preprocessor = ImagePreprocessor(get_params(video))
    start_time = time.time()
    learned = False
    saved = False
    out_writer = cv2.VideoWriter(os.path.join(PATH, 'output_videos', video), cv2.VideoWriter_fourcc(*'mp4v'), 30, (800, 600))
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            fps_start_time = time.time()
            frame = image_preprocessor.pipeline(frame,
                    image_preprocessor.resize_image,
                    image_preprocessor.change_contrast_and_brightness,
                )
            detected, results = detect.detection(frame)
            
            # Draw bounding boxes
            try:
                start_point = int(results[0]['x'] - results[0]['width']//2), int(results[0]['y'] - results[0]['height']//2)
                end_point = int(results[0]['x'] + results[0]['width']//2), int(results[0]['y'] + results[0]['height']//2)
                cv2.rectangle(frame, start_point, end_point, (0, 255, 0), 2)
                cv2.putText(frame, f"{results[0]['class']}: {results[0]['confidence']}", (start_point[0], start_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            except (IndexError, TypeError, KeyError):
                pass
            
            # Calculate fps
            fps_end_time = time.time()
            fps_sum += 1/(fps_end_time - fps_start_time)
            count += 1
            
            # Save results in csv:(filename, true_class, pred_class) and add text to video
            if detected and not saved:
                with open('out/results_v1.csv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow([video, true_df['classname'], results[0]['classname']])
                saved = True
            
            # Draw text in video
            if saved:                
                cv2.putText(frame, "Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
            
            # Save video with bounding boxes
            out_writer.write(frame)
        
            # cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    out_writer.release()
    cv2.destroyAllWindows()
    
# Calculate avg fps
print(f"Average fps: {fps_sum/count}")

100%|██████████| 40/40 [08:59<00:00, 13.49s/it]


## Version 2

In [13]:
from utils.image_preprocessing import ImagePreprocessor
from utils.background_remover import BackgroundRemover
from utils.detect import Detect

In [14]:
PATH = "/media/rodri/Files/Datasets/Evaluation videos/Evaluation_V2"
Background_remover = BackgroundRemover()
detect = Detect()

In [ ]:
# Show videos
import cv2
import os
import time
from tqdm import tqdm
import yaml

fps_sum = 0
count = 0


def get_params(video: str) -> dict:
    """ This function returns the parameters of the video.

    Args:
        video (str): The name of the video which will be converted on the config file path.
            Example: "I_CIDIMEC_Background_1.mp4" -> "config/inside cidimec.yml"

    Returns:
        dict: The parameters of the video.
    """
    place, background, _ = video.split("_")
    
    place = "inside" if place == "I" else "outside"
    
    scenario = f"{place} {background}"
    
    with open(f"config/{scenario}.yml", 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
        
    return cfg

for video in tqdm(true_df['filename']):
    cap = cv2.VideoCapture(os.path.join(PATH, video))
    image_preprocessor = ImagePreprocessor(get_params(video))
    start_time = time.time()
    learned = False
    saved = False
    out_writer = cv2.VideoWriter(os.path.join(PATH, 'output_videos', video), cv2.VideoWriter_fourcc(*'mp4v'), 30, (800, 600))
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            fps_start_time = time.time()
            
            frame = image_preprocessor.pipeline(frame,
                    image_preprocessor.resize_image,
                    image_preprocessor.change_contrast_and_brightness,
                )
            if time.time() - start_time <= 5:
                print("Learning...")
                Background_remover.learn_background(frame)
                continue
            if not learned:
                Background_remover.set_static_background()
                print("Static background set")
                print(f"Background learned in {time.time() - start_time} seconds")
                learned = True
                
            no_bg_frame = Background_remover.remove_background(frame)
            
            detected, results = detect.detection(no_bg_frame)
            
            # Draw bounding boxes
            try:
                start_point = int(results[0]['x'] - results[0]['width']//2), int(results[0]['y'] - results[0]['height']//2)
                end_point = int(results[0]['x'] + results[0]['width']//2), int(results[0]['y'] + results[0]['height']//2)
                cv2.rectangle(frame, start_point, end_point, (0, 255, 0), 2)
                cv2.putText(frame, f"{results[0]['class']}: {results[0]['confidence']}", (start_point[0], start_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            except (IndexError, TypeError, KeyError):
                pass
            
            # Calculate fps
            fps_end_time = time.time()
            fps_sum += 1/(fps_end_time - fps_start_time)
            count += 1
            
            # Save results in csv:(filename, true_class, pred_class) and add text to video
            if detected and not saved:
                with open('out/results_v2.csv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow([video, true_df['classname'], results[0]['classname']])
                saved = True
            
            # Draw text in video
            if saved:                
                cv2.putText(frame, "Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
            
            # Save video with bounding boxes
            out_writer.write(frame)
        
            # cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    out_writer.release()
    cv2.destroyAllWindows()

## Getting metrics

In [ ]:
df_v1 = pd.read_csv('out/results_v1.csv', names=['filename', 'true_class', 'pred_class'])
df_v2 = pd.read_csv('out/results_v2.csv', names=['filename', 'true_class', 'pred_class'])

In [16]:
from sklearn.metrics import classification_report
from seaborn import heatmap
from matplotlib import pyplot as plt

In [ ]:
# Report V1
print(classification_report(df_v1['true_class'], df_v1['pred_class']))

In [ ]:
# Report V2
print(classification_report(df_v2['true_class'], df_v2['pred_class']))

In [ ]:
# Confusion matrix V1
# Confusion matrix percentage
confusion_matrix_percentage = pd.crosstab(df_v1['true_class'], df_v1['predicted_class'], rownames=['True'], colnames=['Predicted'], margins=True, normalize='index')
plt.figure(figsize=(8, 8))

# Change axes by classnames: ['Background', 'Gun, 'Knife']
confusion_matrix_percentage = confusion_matrix_percentage.rename(columns={0: 'Background', 1: 'Gun', 2: 'Knife', 3: 'All'})
confusion_matrix_percentage = confusion_matrix_percentage.rename(index={0: 'Background', 1: 'Gun', 2: 'Knife', 3: 'All'})

heatmap(confusion_matrix_percentage, annot=True, fmt='.2%', cmap='Blues')

In [ ]:
# Confusion matrix V2
# Confusion matrix percentage
confusion_matrix_percentage = pd.crosstab(df_v2['true_class'], df_v2['predicted_class'], rownames=['True'], colnames=['Predicted'], margins=True, normalize='index')
plt.figure(figsize=(8, 8))

# Change axes by classnames: ['Background', 'Gun, 'Knife']
confusion_matrix_percentage = confusion_matrix_percentage.rename(columns={0: 'Background', 1: 'Gun', 2: 'Knife', 3: 'All'})
confusion_matrix_percentage = confusion_matrix_percentage.rename(index={0: 'Background', 1: 'Gun', 2: 'Knife', 3: 'All'})

heatmap(confusion_matrix_percentage, annot=True, fmt='.2%', cmap='Blues')